In [ ]:
### To speed this up, need to move it to scrapy

# Scraper Method

In [19]:
import sys
sys.stdout.write('test')
sys.stdout.write('test')

testtest

In [6]:
import urllib.request
import json
from collections import defaultdict
import random
import time
from datetime import datetime, timedelta

ownerText = 'owner'
shortcodeText = 'shortcode'
timeText = 'taken_at_timestamp'

def getInstagramPosts(startTime, endTime = False):   # Need start time to say if the post if eligible 
    # Next page: https://www.instagram.com/explore/tags/mightcoulddrawtoday/?__a=1&max_id='endcursor'
    # graphql -> hashtag -> edge_hashtag_to_media ->page_info -> end_cursor
    
    raw_posts = [] #Array to hold all posts
    current_posts = []
    tag = 'mightcoulddrawtoday'
    count = None
    end_point = ''
    step = 0

    print("Grabbing posts from online...")
    print("Step: ", end=' ')
    while True:
        try: 
            step += 1
            url = 'https://www.instagram.com/explore/tags/' + tag + '/?__a=1'
            if end_point: url += '&max_id=' + end_point

            print(step, end=', ')

            #Get the data from the site
            raw_json = json.loads(urllib.request.urlopen(url).read().decode())

            count = raw_json['graphql']['hashtag']['edge_hashtag_to_media']['count']

            raw_posts += raw_json['graphql']['hashtag']['edge_hashtag_to_media']['edges']

            #Get the end_point, i.e. the next set of images
            end_point = raw_json['graphql']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']
            if not end_point: break #break if this is there isn't another endpoint
        except Exception as e:
            print(e)
            delay = 60
            print("Sleeping", delay, "seconds")
            time.sleep(60)
    print('Complete')
    
    print('Processing posts...')
    for post in raw_posts:
        if post['node'][timeText] >= startTime:
            if endTime == False or post['node'][timeText] < endTime:
                owner = post['node'][ownerText]['id']
                shortcode = post['node'][shortcodeText]
                current_posts.append({ownerText:owner, shortcodeText:shortcode})
    # print(json.dumps(all_posts,indent=2))
    total_posts=count
    
    print('Listed number of posts:', total_posts)
    print('Total captured posts:', len(raw_posts))
    print('Current captured posts:', len(current_posts))
    
    return current_posts, total_posts

def getOwnerPosts(owner, posts):
    owner_str = str(owner)
    return [post[shortcodeText] for post in posts if post[ownerText] in owner_str]

def getOwnerFrequencies(_dict, key):
    returnDict = defaultdict(int)
    for d in _dict:
        returnDict[d[key]] += 1
    return returnDict

def getEligibleOwners(owner_freq_dict, min_posts=7):
    eligible_owners = []
    for owner_id in owner_freq_dict:
        if owner_freq_dict[owner_id] >= min_posts:
            eligible_owners.append(owner_id)
    return eligible_owners

In [ ]:
### Get new posts from the internet since the start time
startDate = datetime(2018,6,18) #This gives 2/19 as local time
startTime = time.mktime(startDate.timetuple()) # Set this to be either Monday or the time the email sent

endDate = datetime(2018,6,25) #This gives 2/19 as local time
endTime = time.mktime(endDate.timetuple()) # Set this to be either Monday or the time the email sent

all_posts, total_posts = getInstagramPosts(startTime, endTime)

In [ ]:
# Make dictionary of owners frequencies
owner_freq_dict = getOwnerFrequencies(all_posts, ownerText)

# Make list of eligible owners
eligible_owners = getEligibleOwners(owner_freq_dict, min_posts=7)

# Get the winner
winner = random.choice(eligible_owners)

# Get the winner's posts
winner_posts = getOwnerPosts(winner, all_posts)

print('Artists sharing this week: ', len(owner_freq_dict))
print('Total posts shared this week:', len(all_posts))
print('Total posts shared all time:', total_posts)
print('Artists that completed the challenge:', len(eligible_owners))
print('Winner:', winner)
print('posts:')
for post in winner_posts:
    print ('https://www.instagram.com/p/' + post + '/')

In [ ]:
owner_freq_dict
owner_freq_freq = defaultdict(int)
for d in owner_freq_dict:
    owner_freq_freq[owner_freq_dict[d]] += 1
owner_freq_freq

In [ ]:
all_posts[-1]

## Python code to check tags in Instagram

In [ ]:
## Attempt #3
import requests
import json
from bs4 import BeautifulSoup

CLIENT_ID = '576b4683462e4f5e9a31f2ea637d9249'
CLIENT_SECRET='bc74451779a84f8c88d2d26dc200dea2'
REDIRECT_URI='https://www.etymologyexplorer.com/instagramredirect'

GRANT_TYPE = 'authorization_code'
url = 'https://api.instagram.com/oauth/access_token'

session = requests.session()    # use session to handle cookies

In [ ]:

OAuthURL = 'https://api.instagram.com/oauth/authorize/?client_id='+CLIENT_ID+'&redirect_uri='+REDIRECT_URI+'&response_type=code'
# OR GO TO THIS URL:
#https://api.instagram.com/oauth/authorize/?client_id=576b4683462e4f5e9a31f2ea637d9249&redirect_uri=https://www.etymologyexplorer.com/instagramredirect&response_type=code


OAuth_AccessRequest = session.get(OAuthURL)

soup = BeautifulSoup(OAuth_AccessRequest.content,"lxml")
form = soup.form
login_data = {form.input.attrs['name'] : form.input['value']}
login_data.update({'username': 'declannishiyama', 'password': '7Us%TnL7GAbo'})
headers = {'Referer': OAuth_AccessRequest.url}
login_url = 'https://instagram.com{}'.format(form.attrs['action'])
response = session.post(login_url, data=login_data, headers=headers)
soup = BeautifulSoup(response.content, 'lxml')

code = soup.input['value'] #Assuming that the returns first "input" is the secret code
print(soup)
print("Code:", code)



In [ ]:
code = 'dfc70abf07f5409fafccf22ea9de9e05'

data= {'client_id': CLIENT_ID, 
       'client_secret': CLIENT_SECRET,
       'grant_type': GRANT_TYPE,
       'redirect_uri': REDIRECT_URI, 
       'code': code,
       'scope': 'public_content'}

headers = {'Referer': url}
r = session.post(url, data=data, headers=headers)
print(r.content)

access_token = json.loads(r.content)['access_token']
print(access_token)

# Scraper